# Map SNOWPACK points to drainage basins and ice sheets
This notebook provides a mapping between points for which SNOWPACK was run and the drainage basins and ice sheets they belong to.

In [ ]:
import re
import numpy as np
import fiona
import pyproj

from shapely.geometry import Polygon, Point, shape
from shapely.ops import transform
import shapely.prepared

# Initialize required projections
wgs84 = pyproj.CRS('EPSG:4326')  # WGS 84
aps = pyproj.CRS('EPSG:3031')    # Antarctic Polar Stereographic
gps = pyproj.CRS('EPSG:3413')    # Greenland Polar Stereographic

project_to_aps = pyproj.Transformer.from_crs(wgs84, aps, always_xy=True).transform
project_to_gps = pyproj.Transformer.from_crs(wgs84, gps, always_xy=True).transform

## Read SNOWPACK points
SNOWPACK.pts should contain `latitude longitude` for all SNOWPACK run points

In [ ]:
# Read points for which we performed SNOWPACK simulations
SP_pts = np.loadtxt("SNOWPACK.pts", comments="#", delimiter=" ", unpack=False)

# Now transform coordinates
SP_pts_A = []    # WGS 84
SP_pts_G = []    # WGS 84
SP_pts_AT = []   # Antarctic Polar Stereographic
SP_pts_GT = []   # Greenland Polar Stereographic
for i in SP_pts:
    pt = Point(i[1], i[0])                  # WGS-84, EPSG:4326
    if i[0] < 0:                            # Southern hemisphere
        SP_pts_t = transform(project_to_aps, Point(i[1], i[0]))
        SP_pts_A.append([i[1], i[0]])
        SP_pts_AT.append([SP_pts_t.x, SP_pts_t.y])
    else:
        SP_pts_t = transform(project_to_gps, Point(i[1], i[0]))
        SP_pts_G.append([i[1], i[0]])
        SP_pts_GT.append([SP_pts_t.x, SP_pts_t.y])
SP_pts_A = np.array(SP_pts_A)
SP_pts_AT = np.array(SP_pts_AT)
SP_pts_G = np.array(SP_pts_G)
SP_pts_GT = np.array(SP_pts_GT)

## Read in drainage basins and icesheet boundaries
These files were obtained from http://imbie.org/imbie-3/drainage-basins/ and https://earth.gsfc.nasa.gov/cryo/data/polar-altimetry/antarctic-and-greenland-drainage-systems.

In [ ]:
# Rignot
ais_basins_shapefile="./rignot/ANT_Basins_IMBIE2_v1.6.shp"                            # in Antarctic Polar Stereographic
ais_icesheets_shapefile="./rignot/ANT_IceSheets_IMBIE2/ANT_IceSheets_IMBIE2_v1.6.shp" # in Antarctic Polar Stereographic
gris_basins_shapefile="./rignot/GRE_Basins_IMBIE2_v1.3.shp"                           # in EPSG:4326
gris_icesheets_shapefile="./rignot/GRE_IceSheet_IMBIE2/GRE_IceSheet_IMBIE2_v1.shp"    # in EPSG:4326

# Zwally
ais_basins_shapefile_zwally="./zwally/ant_grounded_drainagesystem_polygons.txt"       # in EPSG:4326
ais_icesheets_shapefile_zwally="./zwally/Ant_Grounded_IceSheets_Polygon-1.txt"        # in EPSG:4326
gris_basins_shapefile_zwally="./zwally/grndrainagesystems_ekholm.txt"                 # in EPSG:4326
gris_icesheets_shapefile_zwally="./zwally/GrnIceSheet_Ekholm-1.txt"                   # in EPSG:4326

# Mappings:
AIS = ["None" for i in range(31)]
AIS[28]="AP"
AIS[29]="WAIS"
AIS[30]="EAIS"
# Antarctica basins in Zwally labeled 1 thru 27
AB_labels_zw = [i for i in range(1,28)]
# Greenland basins in Zwally labeled as follows (in order they appear in the file):
GB_labels_zw = [1.1,1.2,1.3,1.4,2.1,2.2,3.1,3.2,3.3,4.1,4.2,4.3,5.0,6.1,6.2,7.1,7.2,8.1,8.2]

### Read in Zwally delineations into shapely Polygons

In [ ]:
#
# Antarctica (note, we convert polygons to EPSG:3031, to be able to properly use .contain())
# Since the South Pole is contained in the polygons, the polygon wraps around the South Pole.
#

# Ice sheets
AIS_pl_zw = ["None" for i in range(31)]   # Initialize array
for i in range(28,31):                    # 28-30: See mappings above
    f = open(ais_icesheets_shapefile_zwally, "r")
    # Skip header
    for l in f:
        if l == "END OF HEADER\n":
            break
    # Read polygon
    pl = []
    for l in f:
        l_s = re.split(' +', l.replace("\n", "").replace("\t", " ").lstrip().rstrip())
        if len(l_s)==3 and int(l_s[2])==i:
            pl.append([float(l_s[1]), float(l_s[0])])
    f.close()
    AIS_pl_zw[i]=transform(project_to_aps, Polygon(pl))   # Convert to polar stereographic

# Drainage basins
AB_pl_zw = ["None" for i in range(28)]   # Initialize array
for i in range(1,28):                    # Drainage basins IDs range from 1-27
    f = open(ais_basins_shapefile_zwally, "r")
    # Skip header
    for l in f:
        if l == "END OF HEADER\n":
            break
    # Read polygon
    pl = []
    for l in f:
        l_s = re.split(' +', l.replace("\n", "").replace("\t", " ").lstrip().rstrip())
        if len(l_s)==3 and int(l_s[2])==i:
            pl.append([float(l_s[1]), float(l_s[0])])
    f.close()
    AB_pl_zw[i]=transform(project_to_aps, Polygon(pl))   # Convert to polar stereographic

In [ ]:
#
# Greenland
#

# Ice sheets
GrIS_pl_zw = "None"
f = open(gris_icesheets_shapefile_zwally, "r")
pl = []
for l in f:
    # Skip header
    if l == "END OF HEADER\n":
        break
# Read polygon
for l in f:
    l_s = re.split(' +', l.replace("\n", "").replace("\t", " ").lstrip().rstrip())
    if len(l_s)==3 and int(l_s[0])==9:
        pl.append([float(l_s[2])-360., float(l_s[1])])
f.close()
GrIS_pl_zw=transform(project_to_gps, Polygon(pl))       # Convert to polar stereographic

# Drainage basins
GB_pl_zw = ["None" for i in range(len(GB_labels_zw))]   # Initialize array
for i in range(len(GB_labels_zw)):                      # Loop over label array
    f = open(gris_basins_shapefile_zwally, "r")
    # Skip header
    for l in f:
        if l == "END OF HEADER\n":
            break
    # Read polygon
    pl = []
    for l in f:
        l_s = re.split(' +', l.replace("\n", "").replace("\t", " ").lstrip().rstrip())
        if len(l_s)==3 and float(l_s[0])==GB_labels_zw[i]:
            pl.append([float(l_s[2])-360., float(l_s[1])])
    f.close()
    GB_pl_zw[i]=transform(project_to_gps, Polygon(pl))  # Convert to polar stereographic

## Loop over SNOWPACK points and list in which drainage basin and ice sheet they are located

In [ ]:
# Print header
print("#lat lon Ant_R/Gr_R Ant_IS_R Ant_ISSub_R Ant_Basin_R Ant_BasinSub_R Gr_IS_R Gr_Basin_R Ant_Z/Gr_R_Z Ant_IS_Z Ant_ISSub_Z Ant_Basin_Z Ant_BasinSub_Z Gr_IS_Z Gr_Basin_Z")
print("# *_R = Rignot, *_Z = Zwally")
# Loop over SNOWPACK points
for i in SP_pts:
    pt = Point(i[1], i[0])                  # WGS-84, EPSG:4326
    pt_aps = transform(project_to_aps, pt)  # Antarctic Polar Stereographic, EPSG: 3031
    pt_gps = transform(project_to_gps, pt)  # Antarctic Polar Stereographic, EPSG: 3413

    #
    # Rignot
    #

    # Initialize info for Antarctica
    Ant = False
    regAB = None      # Region Antarctic Basins
    subregAB = None   # Subregion Antarctic Basins
    regAIS = None     # Region Antarctic Icesheet
    subregAIS = None  # Subregion Antarctic Icesheet
    # Initialize info for Greenland
    Gr = False
    regGB = None      # Region Greenland Basins
    regGIS = None     # Region Greenland Icesheet

    # Check Antarctica
    # Icesheets
    for f in fiona.open(ais_icesheets_shapefile):
        g=shape(f['geometry'])
        if g.contains(pt_aps):
            Ant = True
            regAIS=f['properties']['Regions']
            subregAIS=f['properties']['Subregion']
    # Drainage Basins
    for f in fiona.open(ais_basins_shapefile):
        g=shape(f['geometry'])
        if g.contains(pt_aps):
            Ant = True
            regAB = f['properties']['Regions']
            subregAB = f['properties']['Subregion']

    # Check Greenland
    # Icesheets
    for f in fiona.open(gris_icesheets_shapefile):
        g=shape(f['geometry'])
        if g.contains(pt):
            Gr = True
            regGIS = f['properties']['SUBREGION1']
    # Drainage Basins
    for f in fiona.open(gris_basins_shapefile):
        g=shape(f['geometry'])
        if g.contains(pt):
            Gr = True
            regGB = f['properties']['SUBREGION1']

    # print info
    print(i[0], i[1], end = '')
    if Ant:
        print(' Ant_R', end = '')
    elif Gr:
        print(' Gr_R', end = '')
    else:
        print(' None', end = '')
    print('', regAIS, subregAIS, regAB, subregAB, regGIS, regGB, end = '')


    #
    # Zwally
    #

    # Initialize variables for Antarctica
    Ant = False
    regAB = None      # Region Antarctic Basins
    subregAB = None   # Subregion Antarctic Basins
    regAIS = None     # Region Antarctic Icesheet
    subregAIS = None  # Subregion Antarctic Icesheet
    # Initialize variables for Greenland
    Gr = False
    regGB = None      # Region Greenland Basins
    regGIS = None     # Region Greenland Icesheet

    # Check Antarctica
    # Icesheets
    for k in range(28, 31):  # Loop over the Antarctic Icesheets
        if AIS_pl_zw[k].contains(pt_aps):
            Ant = True
            regAIS=AIS[k]
            subregAIS="None"
    # Drainage Basins
    for k in range(1,28):    # Loop over the Antarctic Drainage Basins
        if AB_pl_zw[k].contains(pt_aps):
            Ant = True
            regAB=k
            subregAB="None"

    # Check Greenland
    # Icesheets
    if GrIS_pl_zw.contains(pt_gps):
        Gr = True
        regGIS = "GrIS"
    # Drainage Basins
    for k in range(len(GB_labels_zw)):              # Loop over label array
        if GB_pl_zw[k].contains(pt_gps):
            Gr = True
            regGB=GB_labels_zw[k]

    # print info
    if Ant:
        print(' Ant_Z', end = '')
    elif Gr:
        print(' Gr_Z', end = '')
    else:
        print(' None', end = '')
    print('', regAIS, subregAIS, regAB, subregAB, regGIS, regGB)

# Determine the area represented by a SNOWPACK grid point based on nearest neighbor interpolation
Here, we create an array representing 1kmx1km Antarctica/Greenland, and we apply nearest neighbor using the SNOWPACK grid points to identify which SNOWPACK grid point is closest to each of the 1kmx1km cells. Counting the number of cells each SNOWPACK grid point represents gives us the area in km2, which can be used to weigh the SNOWPACK simulation to calculate ice sheet/basin wide statistics.

In [ ]:
from scipy.interpolate import NearestNDInterpolator

In [ ]:
#
# Antarctica
#
x = SP_pts_AT[:,0]                       # Select all X-coordinates from SNOWPACK-run points
y = SP_pts_AT[:,1]                       # Select all Y-coordinates from SNOWPACK-run points
z = [i for i in range(len(SP_pts_AT))]   # The z-value is set as the index of the grid point


#
# Rignot
#

# Icesheets
print("#Ant_IS_R Ant_ISSub_R lon lat easting northing area")           # Print header
for f in fiona.open(ais_icesheets_shapefile):
    # Get outer boundary of geometry
    c = shape(f['geometry']).bounds
    XX = np.arange(int(c[0]/1000)*1000, int(c[2]/1000+1)*1000, 1000)   # x-coordinates, km resolution
    YY = np.arange(int(c[1]/1000)*1000, int(c[3]/1000+1)*1000, 1000)   # y-coordinates, km resolution
    X, Y = np.meshgrid(XX, YY)                                         # 2D grid for interpolation
    interp = NearestNDInterpolator(list(zip(x, y)), z)                 # Construct interpolator
    Z = interp(X, Y)                                                   # Perform nearest neighbor interpolation
    # Now loop over all grid points which are covered by the geometry
    g = shapely.prepared.prep(shape(f['geometry']))   # Prepare for efficient contain() operations
    n = [0 for i in range(len(SP_pts_AT))]            # To keep track of the counts for each SNOWPACK-run point
    for i in range(len(XX)):
        for j in range(len(YY)):
            if g.contains(Point(XX[i], YY[j])):
                n[Z[j][i]] = n[Z[j][i]] + 1
    regAIS=f['properties']['Regions']
    subregAIS=f['properties']['Subregion']
    # Print result
    for i in range(len(SP_pts_AT)):
        if n[i] > 0:                 # Only list points that cover the geometry
            print(regAIS, subregAIS, SP_pts_A[i][0], SP_pts_A[i][1], SP_pts_AT[i][0], SP_pts_AT[i][1], n[i])

# Drainage Basins
print("#Ant_Basin_R Ant_BasinSub_R lon lat easting northing area")     # Print header
for f in fiona.open(ais_basins_shapefile):
    # Get outer boundary of geometry
    c = shape(f['geometry']).bounds
    XX = np.arange(int(c[0]/1000)*1000, int(c[2]/1000+1)*1000, 1000)   # x-coordinates, km resolution
    YY = np.arange(int(c[1]/1000)*1000, int(c[3]/1000+1)*1000, 1000)   # y-coordinates, km resolution
    X, Y = np.meshgrid(XX, YY)                                         # 2D grid for interpolation
    interp = NearestNDInterpolator(list(zip(x, y)), z)                 # Construct interpolator
    Z = interp(X, Y)                                                   # Perform nearest neighbor interpolation
    # Now loop over all grid points which are covered by the geometry
    g=shapely.prepared.prep(shape(f['geometry']))
    n = [0 for i in range(len(SP_pts_AT))]            # To keep track of the counts for each SNOWPACK-run point
    for i in range(len(XX)):
        for j in range(len(YY)):
            if g.contains(Point(XX[i], YY[j])):
                n[Z[j][i]] = n[Z[j][i]] + 1
    regAB = f['properties']['Regions']
    subregAB = f['properties']['Subregion']
    # Print result
    for i in range(len(SP_pts_AT)):
        if n[i] > 0:                 # Only list points that cover the geometry
            print(regAB, subregAB, SP_pts_A[i][0], SP_pts_A[i][1], SP_pts_AT[i][0], SP_pts_AT[i][1], n[i])


#
# Zwally
#

# Icesheets
print("#Ant_IS_Z Ant_ISSub_Z lon lat easting northing area")           # Print header
for k in range(28, 31):  # Loop over the Antarctic Icesheets
    # Get outer boundary of geometry
    c = AIS_pl_zw[k].bounds
    XX = np.arange(int(c[0]/1000)*1000, int(c[2]/1000+1)*1000, 1000)   # x-coordinates, km resolution
    YY = np.arange(int(c[1]/1000)*1000, int(c[3]/1000+1)*1000, 1000)   # y-coordinates, km resolution
    X, Y = np.meshgrid(XX, YY)                                         # 2D grid for interpolation
    interp = NearestNDInterpolator(list(zip(x, y)), z)                 # Construct interpolator
    Z = interp(X, Y)                                                   # Perform nearest neighbor interpolation
    # Now loop over all grid points which are covered by the geometry
    g = shapely.prepared.prep(AIS_pl_zw[k])           # Prepare for efficient contain() operations
    n = [0 for i in range(len(SP_pts_AT))]            # To keep track of the counts for each SNOWPACK-run point
    for i in range(len(XX)):
        for j in range(len(YY)):
            if g.contains(Point(XX[i], YY[j])):
                n[Z[j][i]] = n[Z[j][i]] + 1
    regAIS=AIS[k]
    subregAIS="None"
    # Print result
    for i in range(len(SP_pts_AT)):
        if n[i] > 0:                 # Only list points that cover the geometry
            print(regAIS, subregAIS, SP_pts_A[i][0], SP_pts_A[i][1], SP_pts_AT[i][0], SP_pts_AT[i][1], n[i])

# Drainage Basins
print("#Ant_Basin_Z Ant_BasinSub_Z lon lat easting northing area")     # Print header
for k in range(1,28):    # Loop over the Antarctic Drainage Basins    
    # Get outer boundary of geometry
    c = AB_pl_zw[k].bounds
    XX = np.arange(int(c[0]/1000)*1000, int(c[2]/1000+1)*1000, 1000)   # x-coordinates, km resolution
    YY = np.arange(int(c[1]/1000)*1000, int(c[3]/1000+1)*1000, 1000)   # y-coordinates, km resolution
    X, Y = np.meshgrid(XX, YY)                                         # 2D grid for interpolation
    interp = NearestNDInterpolator(list(zip(x, y)), z)                 # Construct interpolator
    Z = interp(X, Y)                                                   # Perform nearest neighbor interpolation
    # Now loop over all grid points which are covered by the geometry
    g=shapely.prepared.prep(AB_pl_zw[k])
    n = [0 for i in range(len(SP_pts_AT))]            # To keep track of the counts for each SNOWPACK-run point
    for i in range(len(XX)):
        for j in range(len(YY)):
            if g.contains(Point(XX[i], YY[j])):
                n[Z[j][i]] = n[Z[j][i]] + 1
    regAB=k
    subregAB="None"
    # Print result
    for i in range(len(SP_pts_AT)):
        if n[i] > 0:                 # Only list points that cover the geometry
            print(regAB, subregAB, SP_pts_A[i][0], SP_pts_A[i][1], SP_pts_AT[i][0], SP_pts_AT[i][1], n[i])

In [ ]:
#
# Greenland
#
x = SP_pts_GT[:,0]                       # Select all X-coordinates from SNOWPACK-run points
y = SP_pts_GT[:,1]                       # Select all Y-coordinates from SNOWPACK-run points
z = [i for i in range(len(SP_pts_GT))]   # The z-value is set as the index of the grid point


#
# Rignot
#

# Icesheets
print("#Gr_IS_R lon lat easting northing area")                        # Print header
for f in fiona.open(gris_icesheets_shapefile):
    # Transform feature to stereographic
    fT = transform(project_to_gps, shape(f['geometry']))
    # Get outer boundary of geometry
    c = fT.bounds
    XX = np.arange(int(c[0]/1000)*1000, int(c[2]/1000+1)*1000, 1000)   # x-coordinates, km resolution
    YY = np.arange(int(c[1]/1000)*1000, int(c[3]/1000+1)*1000, 1000)   # y-coordinates, km resolution
    X, Y = np.meshgrid(XX, YY)                                         # 2D grid for interpolation
    interp = NearestNDInterpolator(list(zip(x, y)), z)                 # Construct interpolator
    Z = interp(X, Y)                                                   # Perform nearest neighbor interpolation
    # Now loop over all grid points which are covered by the geometry
    g = shapely.prepared.prep(fT)   # Prepare for efficient contain() operations
    n = [0 for i in range(len(SP_pts_GT))]            # To keep track of the counts for each SNOWPACK-run point
    for i in range(len(XX)):
        for j in range(len(YY)):
            if g.contains(Point(XX[i], YY[j])):
                n[Z[j][i]] = n[Z[j][i]] + 1
    regGIS = f['properties']['SUBREGION1']
    # Print result
    for i in range(len(SP_pts_GT)):
        if n[i] > 0:                 # Only list points that cover the geometry
            print(regGIS, SP_pts_G[i][0], SP_pts_G[i][1], SP_pts_GT[i][0], SP_pts_GT[i][1], n[i])

# Drainage Basins
print("#Gr_Basin_R lon lat easting northing area")                     # Print header
for f in fiona.open(gris_basins_shapefile):
    # Transform feature to stereographic
    fT = transform(project_to_gps, shape(f['geometry']))
    # Get outer boundary of geometry
    c = fT.bounds
    XX = np.arange(int(c[0]/1000)*1000, int(c[2]/1000+1)*1000, 1000)   # x-coordinates, km resolution
    YY = np.arange(int(c[1]/1000)*1000, int(c[3]/1000+1)*1000, 1000)   # y-coordinates, km resolution
    X, Y = np.meshgrid(XX, YY)                                         # 2D grid for interpolation
    interp = NearestNDInterpolator(list(zip(x, y)), z)                 # Construct interpolator
    Z = interp(X, Y)                                                   # Perform nearest neighbor interpolation
    # Now loop over all grid points which are covered by the geometry
    g=shapely.prepared.prep(fT)
    n = [0 for i in range(len(SP_pts_GT))]            # To keep track of the counts for each SNOWPACK-run point
    for i in range(len(XX)):
        for j in range(len(YY)):
            if g.contains(Point(XX[i], YY[j])):
                n[Z[j][i]] = n[Z[j][i]] + 1
    regGB = f['properties']['SUBREGION1']
    # Print result
    for i in range(len(SP_pts_GT)):
        if n[i] > 0:                 # Only list points that cover the geometry
            print(regGB, SP_pts_G[i][0], SP_pts_G[i][1], SP_pts_GT[i][0], SP_pts_GT[i][1], n[i])


#
# Zwally
#

# Icesheets
print("#Gr_IS_Z lon lat easting northing area")                    # Print header
# Get outer boundary of geometry
c = GrIS_pl_zw.bounds
XX = np.arange(int(c[0]/1000)*1000, int(c[2]/1000+1)*1000, 1000)   # x-coordinates, km resolution
YY = np.arange(int(c[1]/1000)*1000, int(c[3]/1000+1)*1000, 1000)   # y-coordinates, km resolution
X, Y = np.meshgrid(XX, YY)                                         # 2D grid for interpolation
interp = NearestNDInterpolator(list(zip(x, y)), z)                 # Construct interpolator
Z = interp(X, Y)                                                   # Perform nearest neighbor interpolation
# Now loop over all grid points which are covered by the geometry
g = shapely.prepared.prep(GrIS_pl_zw)             # Prepare for efficient contain() operations
n = [0 for i in range(len(SP_pts_GT))]            # To keep track of the counts for each SNOWPACK-run point
for i in range(len(XX)):
    for j in range(len(YY)):
        if g.contains(Point(XX[i], YY[j])):
            n[Z[j][i]] = n[Z[j][i]] + 1
regGIS = "GrIS"
# Print result
for i in range(len(SP_pts_GT)):
    if n[i] > 0:                 # Only list points that cover the geometry
        print(regGIS, SP_pts_G[i][0], SP_pts_G[i][1], SP_pts_GT[i][0], SP_pts_GT[i][1], n[i])

# Drainage Basins
print("#Gr_Basin_Z lon lat easting northing area")                     # Print header
for k in range(len(GB_labels_zw)):              # Loop over label array
    # Get outer boundary of geometry
    c = GB_pl_zw[k].bounds
    XX = np.arange(int(c[0]/1000)*1000, int(c[2]/1000+1)*1000, 1000)   # x-coordinates, km resolution
    YY = np.arange(int(c[1]/1000)*1000, int(c[3]/1000+1)*1000, 1000)   # y-coordinates, km resolution
    X, Y = np.meshgrid(XX, YY)                                         # 2D grid for interpolation
    interp = NearestNDInterpolator(list(zip(x, y)), z)                 # Construct interpolator
    Z = interp(X, Y)                                                   # Perform nearest neighbor interpolation
    # Now loop over all grid points which are covered by the geometry
    g=shapely.prepared.prep(GB_pl_zw[k])
    n = [0 for i in range(len(SP_pts_GT))]            # To keep track of the counts for each SNOWPACK-run point
    for i in range(len(XX)):
        for j in range(len(YY)):
            if g.contains(Point(XX[i], YY[j])):
                n[Z[j][i]] = n[Z[j][i]] + 1
    regGB=GB_labels_zw[k]    # Print result
    for i in range(len(SP_pts_GT)):
        if n[i] > 0:                 # Only list points that cover the geometry
            print(regGB, SP_pts_G[i][0], SP_pts_G[i][1], SP_pts_GT[i][0], SP_pts_GT[i][1], n[i])

In [ ]:
# Plot interpolated field
import matplotlib.pyplot as plt
plt.pcolormesh(X, Y, Z, shading='auto')
plt.plot(x, y, "ok", label="input point")
plt.legend()
plt.colorbar()
plt.axis("equal")
plt.show()